In [16]:
import os
import natsort
import re
import pandas as pd

In [47]:
    models_dirs = os.listdir('remote_experiments')
    models_dirs = [os.path.join('remote_experiments', model_dir) for model_dir in natsort.natsorted(models_dirs, reverse=False)]

    models = {}

    for model_dir in models_dirs:
        model_id = int(model_dir.split('_')[-1])
        trainlogs_dir = os.path.join(model_dir, f'trainlogs_{model_id}.log')
        with open(trainlogs_dir) as file:
            model_vars = {}
            for line in file:
                if ":" in line:
                    line = line.replace('\n','')
                    name, var = line.partition(":")[::2]
                    model_vars[name.strip()] = var.strip()
                else:
                    break

            models[model_id] = model_vars
        



    
    

In [48]:
for model_dir in models_dirs:
    model_id = int(model_dir.split('_')[-1])
    trainlogs_dir = os.path.join(model_dir, f'trainlogs_{model_id}.log')
    with open(trainlogs_dir) as file:
        train_losses = []
        valid_losses = []
        test_losses = []

        train_accuracies = []
        valid_accuracies = []
        test_accuracies = []
        
        for line in file:
            loss = re.search('Average loss: ([\w.]+)', line).group(1) if re.search('Average loss: ([\w.]+)', line) else None
            accuracy = re.search(re.search('Accuracy: ([\w/]+)', line).group(1) + ' \(([\w.]+)', line).group(1) if re.search('Accuracy: ([\w/]+)', line) else None
            if loss and accuracy:
                loss = float(loss)
                accuracy = float(accuracy)
                if "Train" in line:
                    train_losses.append(loss)
                    train_accuracies.append(accuracy)
                elif "Validation" in line:
                    valid_losses.append(loss)
                    valid_accuracies.append(accuracy)
                elif "Test" in line:
                    test_losses.append(loss)
                    test_accuracies.append(accuracy)
                    
    models[model_id]["train_losses"] = train_losses
    models[model_id]["valid_losses"] = valid_losses
    models[model_id]["test_losses"] = test_losses
    
    models[model_id]["train_accuracies"] = train_accuracies
    models[model_id]["valid_accuracies"] = valid_accuracies
    models[model_id]["test_accuracies"] = test_accuracies
    
    models[model_id]["best_train_loss"] = str(round(min(train_losses), 4))
    models[model_id]["best_valid_loss"] = str(round(min(valid_losses), 4))
    models[model_id]["best_test_loss"] = str(round(min(test_losses), 4))
    
    models[model_id]["best_train_accuracy"] = str(round(max(train_accuracies), 2))
    models[model_id]["best_valid_accuracy"] = str(round(max(valid_accuracies), 2))
    models[model_id]["best_test_accuracy"] = str(round(max(test_accuracies), 2)) + ' %'
    
        

In [49]:
with open('prints.txt', 'w') as prints_fd:
    models_by_dataset = dict(sorted(models.items(), key=lambda x: (x[1]['dataset'], x[1]['best_test_accuracy']), reverse=True))
    for model_id, model_vars in zip(models_by_dataset.keys(), models_by_dataset.values()):
        print(f"MODEL {model_id}:\n", file=prints_fd)
        print(f"- Dataset: {model_vars['dataset'].upper()}", file=prints_fd)
        print(f"- Batch Size: {model_vars['batch_size']}", file=prints_fd)
        print(f"- Epochs: {model_vars['epochs']}", file=prints_fd)
        print(f"- Learning Rate: {model_vars['lr']}", file=prints_fd)
        if 'scheduler' in model_vars.keys():
            print(f"- Scheduler: {model_vars['scheduler'].capitalize()}", file=prints_fd)
        else:
            print("- Scheduler: Plateau", file=prints_fd)


    #     print(f")
        print("- Routing Algorithm: " + re.sub(r'(\w)([A-Z])', r'\1 \2', model_vars['routing_module']), file=prints_fd)
        if model_vars['extra_conv'] == "True":
            print("- Extra Convolutional Layer", file=prints_fd)

        if model_vars['routing_module'] == "AgreementRouting":
            print("- Number of Iterations: ", model_vars['routing_iterations'], file=prints_fd)
        elif model_vars['routing_module'] == "WeightedAverageRouting":
            pass
        elif model_vars['routing_module'] == "DropoutWeightedAverageRouting":
            print("- Dropout Probability: ", model_vars['dropout_probability'], file=prints_fd)
        elif model_vars['routing_module'] == "SubsetRouting":
            print("- Subset Fraction: ", model_vars['subset_fraction'], file=prints_fd)
        elif model_vars['routing_module'] == "RansacRouting":
            print("- Number of Hypotheses: ", model_vars['n_hypotheses'], file=prints_fd)
            print("- Subset Fraction: ", model_vars['subset_fraction'], file=prints_fd)

        print(f"- Test Accuracy: {model_vars['best_test_accuracy']}%", file=prints_fd)


        print('\n\n', file=prints_fd)


In [61]:
to_drop  = ['no_cuda', 'seed', 'with_reconstruction', 'padding', 'brightness', 
            'contrast', 'crop_dim', 'num_workers', 'test_affNIST',  'cuda', 'channels',
            'n_classes', 'Hin', 'Win', 'train_losses', 'valid_losses',
            'test_losses', 'train_accuracies', 'valid_accuracies',
            'test_accuracies', 'best_train_loss', 'best_valid_loss',
            'best_test_loss', 'best_train_accuracy', 'best_valid_accuracy', 'routing_iterations', 'load_model_id']



df = pd.DataFrame()

for model_id, model_vars in zip(models_by_dataset.keys(), models_by_dataset.values()):
#     tmp = pd.DataFrame([model_vars], index=[model_id]).rename_axis('Model ID')
    tmp = pd.DataFrame([model_vars], index=[model_id])
    df = df.append(tmp)
df = df.drop(to_drop, axis = 1)


# Ignore useless hyperparameters depending on the Routing Module
df.loc[df["routing_module"] == "WeightedAverageRouting", ("dropout_probability", "subset_fraction", "n_hypotheses", "metric")] = ("-", "-", "-", "-")
df.loc[df["routing_module"] == "DropoutWeightedAverageRouting", ("subset_fraction", "n_hypotheses", "metric")] = ("-", "-", "-")
df.loc[df["routing_module"] == "SubsetRouting", ("dropout_probability", "n_hypotheses")] = ("-", "-")                                                        
df.loc[df["routing_module"] == "RansacRouting", "dropout_probability"] = "-"  

df.loc[(df["routing_module"] == "SubsetRouting") & (pd.isna(df["metric"])), "metric"] = "MSE"
df.loc[(df["routing_module"] == "RansacRouting") & (pd.isna(df["metric"])), "metric"] = "MSE" 

# Set "-" if no Scheduler
df.loc[df["scheduler"] == "None", "scheduler"] = "-"
df.loc[pd.isna(df["scheduler"]), "scheduler"] = "-" 

# Set "-" if no extra capsule layer
df.loc[pd.isna(df["extra_caps"]), "extra_caps"] = "-" 

# # Set "-" if no extra capsule layer
# df.loc[pd.isna(df["extra_caps"]), "extra_caps"] = "-" 

# Place Accuracy column last
test_acc_column = df.pop('best_test_accuracy')
df.insert(df.shape[1], 'best_test_accuracy', test_acc_column)


# Rename routing modules (to make them shorter)
df.loc[df["routing_module"] == "WeightedAverageRouting", "routing_module"] = "WeightedAvg"
df.loc[df["routing_module"] == "DropoutWeightedAverageRouting", "routing_module"] = "Dropout"
df.loc[df["routing_module"] == "SubsetRouting", "routing_module"] = "Subset"                                                        
df.loc[df["routing_module"] == "RansacRouting", "routing_module"] = "Ransac"


# Rename Columns
df.rename(columns = {'best_test_accuracy':'Test Accuracy (%)'}, inplace = True)
df.rename(columns = {'dataset':'Dataset'}, inplace = True)
df.rename(columns = {'batch_size':'Batch Size'}, inplace = True)
df.rename(columns = {'epochs':'Epochs'}, inplace = True)
df.rename(columns = {'lr':'Learning Rate'}, inplace = True)
df.rename(columns = {'routing_module':'Routing Module'}, inplace = True)
df.rename(columns = {'dropout_probability':'Dropout Probability'}, inplace = True)
df.rename(columns = {'scheduler':'Scheduler'}, inplace = True)
df.rename(columns = {'subset_fraction':'Subset Fraction'}, inplace = True)
df.rename(columns = {'n_hypotheses':'Number of Hypotheses'}, inplace = True)
df.rename(columns = {'metric':'Similarity Metric'}, inplace = True)

df.replace('-', u'\u2014', inplace=True)






# **Routing Algorithms**
Έστω $V$ το σύνολο των votes των child capsules (layer $L$) για τα poses των parent capsules (layer $L+1$). O $V$ αποτελείται αποτελείται από $V_{ij}$ vectors που δείχνουν το vote του child capsule $i$ για το pose του $j$ parent capsule. 

Έστω $ V_j = [V_{1j},V_{2j},\ldots,V_{ij},\ldots] $ το σύνολο των votes των child capsules προς τον parent j.



## **Weighted Average Routing**


1. $procedure$ $WeightedAverageRouting$ $(V_j):$

2. $\quad$ $\mu_{j} \leftarrow \frac{\sum_i{\lVert  V_{ij}  \rVert \cdot V_{ij}}}{\sum_i{\lVert  V_{ij}  \rVert}} $ $\quad$ $\quad$ $\quad$

3. $\quad$ $return$ $\mu_{j}$



## **Dropout Weighted Average Routing**


Έστω $p$ η dropout πιθανότητα, δηλαδή η πιθανότητα να απενεργοποιηθεί ένα child capsule vote προς τον parent $j$.

1. $procedure$ $DropoutWeightedAverageRouting$ $(V_j, p):$

2. $\quad$ $r_{j} \leftarrow  [r_{1j},r_{2j},\ldots,r_{ij},\ldots] $, όπου $r_{ij} \thicksim Bernoulli(p) $

3. $\quad$ $U_j \leftarrow r_{j} \cdot V_j$ $\quad \quad$ $\quad$ $($ i.e. $U_h=[0,V_{2j},V_{3j}, 0 \ldots,V_{ij},\ldots]$ $)$

4. $\quad$ $\mu_{j} \leftarrow \frac{\sum_i{\lVert  U_{ij}  \rVert \cdot U_{ij}}}{\sum_i{\lVert  U_{ij}  \rVert}} $ $\quad$ $\quad$ $\quad$

3. $\quad$ $return$ $\mu_{j}$



## **Subset Routing**


Έστω $M$ το πλήθος των child votes $V_{ij}$ που θα ενεργοποιούνται προκειμένου να συνεισφέρουν στον υπολογισμό του pose του parent $j$


1. $procedure$ $SubsetRouting$ $(V_j, M):$

2. $\quad$ $\mu_{j} \leftarrow \frac{\sum_i{\lVert  V_{ij}  \rVert \cdot V_{ij}}}{\sum_i{\lVert  V_{ij}  \rVert}} $ $\quad$ $\quad$ $\quad$

3. $\quad$ $L_{j} \leftarrow  \lVert \mu_{j} -  V_{ij} \rVert $

4. $\quad$ $threshold_j \leftarrow  M_{th}$ smallest loss in $L_j$

5. $\quad$ $r_{j} \leftarrow$ vector with the $i_{th}$ being $1$ if $L_{ij} \leq threshold_j$ and  $0$ otherwise

6. $\quad$ $U_j \leftarrow r_{j} \cdot V_j$ $\quad \quad$ $\quad$ $($ i.e. $U_h=[0,V_{2j},V_{3j}, 0 \ldots,V_{ij},\ldots]$ containing $M$ non-zero vectors $)$

7. $\quad$ $\mu'_{j} \leftarrow \frac{\sum_i{\lVert  U_{ij}  \rVert \cdot U_{ij}}}{\sum_i{\lVert  U_{ij}  \rVert}}$

8. $\quad$ $return$ $\mu'_{j}$



## **Ransac Routing**


Έστω $H$ το πλήθος των hypotheses που θα κάνουμε, και $M$ το πλήθος των child votes $V_{ij}$ που θα ενεργοποιούνται προκειμένου να συνεισφέρουν στον υπολογισμό του pose του parent $j$.


1. $procedure$ $RansacRouting$ $(V_j, H, M):$

2. $\quad$ for $H$ iterations do:

3. $\quad \quad$ $r_{j,h} \leftarrow$ random vector with the $i_{th}$ element is either $1$ or $0$ indicating if the $i_{th}$ capsule vote is activated or not ($M$ $1_s$ in total)

4. $\quad \quad$ $U_h \leftarrow r_{j,h} \cdot V_j$ $\quad \quad$ $\quad$ $($ i.e. $U_h=[0,V_{2j},V_{3j}, 0 \ldots,V_{ij},\ldots]$ containing $M$ non-zero vectors $)$

5. $\quad \quad$ $\mu_{j,h} \leftarrow \frac{\sum_i{\lVert  U_{h,i}  \rVert \cdot U_{h,i}}}{\sum_i{\lVert  U_{h,i}  \rVert}} $ $\quad$ $\quad$ $\quad$ $($ we have to choose the optimal $\mu_{j,h}$ that minimizes the following loss $)$

6. $\quad \quad$ $L_{j,h} \leftarrow  \sum_i{ \lVert \mu_{j,h} -  V_{ij} \rVert} $

<!-- 6. $\quad \quad$ $L_{j,h} \leftarrow - \sum_i{\log{(\lVert  V_{ij}  \rVert)} \lVert \mu_{j,h} -  V_{ij} \rVert} $ -->

7. $\quad$ $h^{*} \leftarrow \arg \min_{h} (L_{j,h})$

8. $\quad$ $U^{*}_j \leftarrow r_{j,h^{*}} \cdot V_j$ $\quad \quad$ $\quad$ $($ i.e. $U^{*}_h=[0,V_{2j},V_{3j}, 0 \ldots,V_{ij},\ldots]$ containing $M$ non-zero vectors $)$

9. $\quad$ $\mu^{*}_{j} \leftarrow \frac{\sum_i{\lVert  U^{*}_{ij}  \rVert \cdot U^{*}_{ij}}}{\sum_i{\lVert  U^{*}_{ij}  \rVert}} $ $\quad$ $\quad$ $\quad$

10. $\quad$ $return$ $\mu^{*}_{j}$


In [62]:
def display_df(df, dataset):
    
    df = df.loc[df['Dataset'] == dataset]
    
    # Add styles
    dfStyler = df.style.set_properties(**{'text-align': 'center'})
    dfStyler = dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    display(dfStyler)
    return
    

In [63]:
datasets = ['mnist', 'smallnorb', 'fashionmnist', 'svhn', 'cifar10']
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
UNDERLINE_END = '\033[0m'

for dataset in datasets:
    print('\n\n\n' + BOLD + UNDERLINE + dataset.upper() + UNDERLINE_END + ":")
    display_df(df, dataset)




MNIST:


,Dataset,Batch Size,Epochs,Learning Rate,Scheduler,extra_conv,extra_caps,Routing Module,Dropout Probability,Number of Hypotheses,Subset Fraction,Similarity Metric,Test Accuracy (%)
7,mnist,512,150,0.001,—,False,—,WeightedAvg,—,—,—,—,99.67 %
13,mnist,512,150,0.001,exponential,False,—,WeightedAvg,—,—,—,—,99.67 %
40,mnist,512,150,0.001,plateau,False,—,Subset,—,—,0.3,MSE,99.66 %
1,mnist,256,150,0.001,—,False,—,Dropout,0.4,—,—,—,99.64 %
30,mnist,512,150,0.001,plateau,False,—,Dropout,0.2,—,—,—,99.63 %
55,mnist,256,150,0.003,exponential,False,—,WeightedAvg,—,—,—,—,99.63 %
25,mnist,512,150,0.001,plateau,False,—,Dropout,0.8,—,—,—,99.62 %
18,mnist,512,150,0.001,plateau,False,—,Dropout,0.6,—,—,—,99.61 %
35,mnist,512,150,0.001,plateau,False,—,Subset,—,—,0.5,MSE,99.6 %
45,mnist,512,150,0.001,plateau,False,—,Subset,—,—,0.7,MSE,99.6 %





SMALLNORB:


,Dataset,Batch Size,Epochs,Learning Rate,Scheduler,extra_conv,extra_caps,Routing Module,Dropout Probability,Number of Hypotheses,Subset Fraction,Similarity Metric,Test Accuracy (%)
46,smallnorb,512,250,0.001,plateau,False,—,Subset,—,—,0.7,MSE,96.04 %
121,smallnorb,512,250,0.001,—,False,False,Subset,—,—,0.7,cosine,95.99 %
107,smallnorb,128,250,0.001,—,False,True,WeightedAvg,—,—,—,—,95.98 %
65,smallnorb,256,150,0.001,—,False,—,Ransac,—,10,0.5,MSE,95.81 %
31,smallnorb,512,250,0.001,plateau,False,—,Dropout,0.2,—,—,—,95.51 %
51,smallnorb,512,250,0.001,—,False,—,Subset,—,—,0.7,MSE,95.42 %
2,smallnorb,256,150,0.001,—,False,—,Dropout,0.4,—,—,—,95.4 %
8,smallnorb,512,150,0.001,—,False,—,WeightedAvg,—,—,—,—,95.32 %
14,smallnorb,512,250,0.001,exponential,False,—,WeightedAvg,—,—,—,—,95.26 %
26,smallnorb,512,250,0.001,plateau,False,—,Dropout,0.8,—,—,—,95.25 %





FASHIONMNIST:


,Dataset,Batch Size,Epochs,Learning Rate,Scheduler,extra_conv,extra_caps,Routing Module,Dropout Probability,Number of Hypotheses,Subset Fraction,Similarity Metric,Test Accuracy (%)
108,fashionmnist,128,400,0.001,—,False,True,WeightedAvg,—,—,—,—,93.84 %
117,fashionmnist,64,400,0.001,—,False,True,WeightedAvg,—,—,—,—,93.74 %
10,fashionmnist,512,400,0.001,—,False,—,WeightedAvg,—,—,—,—,93.4 %
57,fashionmnist,256,1000,0.003,exponential,False,—,WeightedAvg,—,—,—,—,93.38 %
32,fashionmnist,512,500,0.001,plateau,False,—,Dropout,0.2,—,—,—,93.3 %
3,fashionmnist,256,150,0.001,—,False,—,Dropout,0.4,—,—,—,93.23 %
15,fashionmnist,512,500,0.001,exponential,False,—,WeightedAvg,—,—,—,—,93.19 %
20,fashionmnist,512,500,0.001,plateau,False,—,Dropout,0.6,—,—,—,93.18 %
27,fashionmnist,512,500,0.001,plateau,False,—,Dropout,0.8,—,—,—,92.85 %
52,fashionmnist,512,700,0.001,—,False,—,Subset,—,—,0.7,MSE,92.75 %





SVHN:


,Dataset,Batch Size,Epochs,Learning Rate,Scheduler,extra_conv,extra_caps,Routing Module,Dropout Probability,Number of Hypotheses,Subset Fraction,Similarity Metric,Test Accuracy (%)
118,svhn,64,400,0.001,—,False,True,WeightedAvg,—,—,—,—,95.62 %
115,svhn,128,400,0.001,—,False,True,WeightedAvg,—,—,—,—,95.39 %
11,svhn,512,400,0.001,—,False,—,WeightedAvg,—,—,—,—,94.92 %
58,svhn,256,1000,0.003,exponential,False,—,WeightedAvg,—,—,—,—,94.89 %
33,svhn,512,500,0.001,plateau,False,—,Dropout,0.2,—,—,—,94.68 %
4,svhn,256,150,0.001,—,False,—,Dropout,0.4,—,—,—,94.54 %
16,svhn,512,500,0.001,exponential,False,—,WeightedAvg,—,—,—,—,94.5 %
21,svhn,512,500,0.001,plateau,False,—,Dropout,0.6,—,—,—,94.37 %
53,svhn,512,700,0.001,—,False,—,Subset,—,—,0.7,MSE,94.19 %
62,svhn,256,150,0.001,—,False,—,Ransac,—,10,0.7,MSE,94.13 %





CIFAR10:


,Dataset,Batch Size,Epochs,Learning Rate,Scheduler,extra_conv,extra_caps,Routing Module,Dropout Probability,Number of Hypotheses,Subset Fraction,Similarity Metric,Test Accuracy (%)
119,cifar10,64,400,0.001,—,False,True,WeightedAvg,—,—,—,—,84.65 %
116,cifar10,128,400,0.001,—,False,True,WeightedAvg,—,—,—,—,83.84 %
12,cifar10,512,400,0.001,—,False,—,WeightedAvg,—,—,—,—,82.56 %
17,cifar10,512,500,0.001,exponential,False,—,WeightedAvg,—,—,—,—,82.38 %
59,cifar10,256,1000,0.003,exponential,False,—,WeightedAvg,—,—,—,—,82.16 %
34,cifar10,512,500,0.001,plateau,False,—,Dropout,0.2,—,—,—,82.07 %
5,cifar10,512,500,0.001,—,False,—,Dropout,0.4,—,—,—,81.9 %
22,cifar10,512,500,0.001,plateau,False,—,Dropout,0.6,—,—,—,81.5 %
6,cifar10,512,500,0.001,—,True,—,Dropout,0.4,—,—,—,81.28 %
63,cifar10,256,150,0.001,—,False,—,Ransac,—,10,0.7,MSE,80.67 %
